In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings("ignore")

Loading Data

In [ ]:
train=pd.read_csv('/content/drive/MyDrive/internship/vpods/ASHRAE/train.csv')
test=pd.read_csv('/content/drive/MyDrive/internship/vpods/ASHRAE/test.csv')
building=pd.read_csv('/content/drive/MyDrive/internship/vpods/ASHRAE/building_metadata.csv')
weather_train=pd.read_csv('/content/drive/MyDrive/internship/vpods/ASHRAE/weather_train.csv')
weather_test=pd.read_csv('/content/drive/MyDrive/internship/vpods/ASHRAE/weather_test.csv')

Merging Dataframes

In [ ]:
#mergind dataframe
building_train=pd.merge(building,train,how='inner',on='building_id')
building_train.dropna(inplace=True)
weather_train.dropna(inplace=True)
building_train_weather=pd.merge(building_train,weather_train,how='inner',on=['site_id','timestamp'])

Aggregrate Values

In [ ]:
def agg_values(df,cols):
    for col in cols:
        agg_dict.append({'feature':col,'max':df[col].max(),'min':df[col].min(),'mean':df[col].mean(),'median':df[col].median(),'mode':df[col].mode(),'var':df[col].var(),
                    })
    return agg_dict

In [ ]:
cols=[  'square_feet', 'year_built',
       'floor_count', 'meter', 'meter_reading', 'air_temperature',
       'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr',
       'sea_level_pressure', 'wind_direction', 'wind_speed']
agg_dict=[]
agg_dict=agg_values(df,cols)

Feature engineering

In [ ]:
#detecting outliers
import numpy as np
def remove_outliers(df,col):
    q1=df[col].quantile(0.25)
    q3=df[col].quantile(0.75)
    IQR=q3-q1
    lower_bound=q1-1.5*IQR
    upper_bound=q3+1.5*IQR
    df[col]=np.clip(df[col],lower_bound,upper_bound)
    return df

def out_all(df,col_list):
    for c in col_list:
        df=remove_outliers(df,c)
    return df


#label encoding
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
data['primary_use']=le.fit_transform(data['primary_use'])
print(le.classes_)

cols=[ 'primary_use', 'square_feet', 
       'floor_count',  'meter_reading', 'air_temperature',
       'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr',
       'sea_level_pressure',  'wind_speed']
data=out_all(data,cols)
data.plot(kind='box',subplots=True,layout=(7,3),figsize=(16,20))

cols=[ 'primary_use', 'square_feet', 
       'floor_count',  'meter_reading', 'air_temperature',
       'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr',
       'sea_level_pressure',  'wind_speed']
data=out_all(data,cols)
data.plot(kind='box',subplots=True,layout=(7,3),figsize=(16,20))

from sklearn.preprocessing import StandardScaler

scaler_train=StandardScaler()
X_train=scaler_train.fit_transform(X_train)
scaler_test=StandardScaler()
X_test=scaler_test.fit_transform(X_test)


colm=data.drop(['timestamp','meter_reading'],axis=1).columns
X_train=pd.DataFrame(X_train)
X_train.columns=colm
X_test=pd.DataFrame(X_test)
X_test.columns=colm

feature selection

In [ ]:
#random forest feature importance
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=150)
rf.fit(X_train, y_train)
sort = rf.feature_importances_.argsort()
rf.feature_importances_[sort]
plt.figure(figsize=(20,5))
sns.barplot(X_train.columns[sort],rf.feature_importances_[sort])
plt.plot()

#mutual information regression
from sklearn.feature_selection import mutual_info_regression

mutual_info=mutual_info_regression(X_train,y_train)
mutual_info=pd.Series(mutual_info)
mutual_info.index=X_train.columns
mutual_info.sort_values(ascending=False).plot.bar(figsize=(15,5))

#f_regression
from sklearn.feature_selection import f_regression
f_r=f_regression(X_train,y_train)
f_r=pd.Series(f_r[0])
f_r.index=X_train.columns
f_r.sort_index(ascending=False).plot.bar(figsize=(15,5))
plt.title("f_score")

pv=f_regression(X_train,y_train)
pv=pd.Series(pv[1])
pv.index=X_train.columns
pv.sort_index(ascending=False).plot.bar(figsize=(15,5))
plt.title("p_value")


#selected features
features_selected=[ 'building_id', 'primary_use', 'square_feet', 'year_built',
       'floor_count', 'air_temperature', 
       'dew_temperature',  'sea_level_pressure',
       'wind_direction', 'wind_speed']

X_train=X_train[features_selected]
X_test=X_test[features_selected]



model building

In [ ]:
from sklearn.ensemble import RandomForestRegressor
def random_forest_model(X_train,X_test,y_train,y_test):
    
    rfr = RandomForestRegressor(criterion= 'squared_error', max_features= 'log2',n_estimators = 80)
    rfr.fit(X_train,y_train)
    y_pred=rfr.predict(X_test)
    return y_pred
